In [1]:
# for Exited, 1 is not-exited, -1 is exited
# for gender, male is 0 and female is 1

#clean outlier data from monthly salary (scale or remove?)

#need to do either undersampling or oversampling, about 8000 users stayed while only 2000 left

#standardscaler or create new features from the features (salary, balance, age, credit score, tenure)

# separate data by region? looks like most data is from france. Much less likely for french customers to leave.
# i.e. the leave data for all three countries is similar but the stay data is much higher in france.

#MY ASSUMPTIONS:
#The data was collected at two points in a year. First point gave balance, salary, etc., then later that year they checked if that person had left or stayed
#Balance may still exist if a person left the bank because it represents the balance they had before leaving
#User is considered an active user if they are still using the bank services in some way. its possible for an active member to leave. its possible for an inactive member to stay.

#is profitable idea: check a 1 for yes and 0 for no, based on iff the user is both an active member AND has a positive balance

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path

from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC


In [3]:

starter_df = pd.read_csv(Path("c://users/ajcth/documents/github/bank_churn_project/Resources/churn.csv"))
starter_df.sample(10)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
5904,5905,15664270,Balsillie,692,Germany,Male,45,6,142084.04,4,1,0,188305.85,1
2751,2752,15720671,Ibezimako,704,France,Male,42,8,129735.30,2,1,1,179565.57,0
1630,1631,15634950,Obiajulu,657,Germany,Male,57,8,107174.58,1,1,1,126369.55,1
9109,9110,15703682,Kalinina,681,Spain,Male,33,10,0.00,1,0,0,158336.36,0
7446,7447,15636626,Morrison,718,France,Male,35,3,97560.16,1,1,1,53511.74,0
7622,7623,15796413,Green,794,France,Male,46,6,0.00,2,1,0,195325.74,0
6531,6532,15675231,Nwankwo,518,France,Female,45,8,0.00,2,1,1,36193.07,0
7320,7321,15778142,Shih,850,Germany,Female,31,1,130089.56,2,1,1,4466.21,0
5349,5350,15567246,Selwyn,684,Germany,Male,32,3,102630.13,2,1,1,127433.47,0
3976,3977,15792180,Chiekwugo,566,Germany,Male,22,7,144954.75,2,1,0,102246.00,0


In [4]:
#Convert the values of the Gender column from "M" or "F" to "0" or "1"
starter_df = starter_df.replace({'Gender' : {'Male': 0, 'Female': 1}})
#Make HasCrCard a -1 if they don't, so it reflects thats a negative
starter_df = starter_df.replace({'HasCrCard' : {1: -1, 0 : 1}})
#Make IsActiveMember a -1 if they arent, so it reflects thats a negative
starter_df = starter_df.replace({'IsActiveMember' : {0: -1}})
#Drop the useless columns
starter_df = starter_df.drop(columns=['RowNumber', 'CustomerId', 'Surname'])
#Drop Nan's
starter_df.dropna(inplace=True)

In [5]:
starter_df.sample(10)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
4567,554,Spain,1,27,4,0.00,2,-1,1,135083.73,0
7941,592,Germany,0,33,2,156570.86,1,-1,1,37140.20,0
8907,786,France,0,34,3,137361.96,1,1,-1,183682.09,0
95,515,Spain,0,35,10,176273.95,1,1,1,121277.78,0
4246,617,France,1,30,8,0.00,1,-1,1,92621.90,0
3006,747,France,1,45,1,114959.12,1,-1,-1,189362.39,1
5979,677,France,1,24,0,148298.59,2,1,-1,182913.95,0
3303,632,France,1,44,3,133793.89,1,-1,1,34607.14,1
8330,815,Spain,1,38,2,48387.00,1,-1,-1,184796.84,0
5333,717,Spain,0,33,1,99106.73,1,1,-1,194467.23,0


In [6]:
#Not sure if I should use this, since im doing feature creation next on these two unscaled variables
scaler = StandardScaler()
starter_df[['ScaledBalance', 'ScaledEstimatedSalary']] = scaler.fit_transform(starter_df[['Balance', 'EstimatedSalary']])

In [7]:
#use feature creation to make a ratio between age and tenure
starter_df['BalanceSalaryRatio'] = (starter_df['Balance'] / starter_df['EstimatedSalary']).round(2)
starter_df.drop(columns=['Balance', 'EstimatedSalary'], inplace=True)

In [8]:
# Create a OneHotEncoder instance
#enc = OneHotEncoder(sparse=False)

# Encode categorcal variables using OneHotEncoder
#encoded_data = enc.fit_transform(starter_df[['Geography']])

In [9]:
#encoded_df_rough = pd.DataFrame(
#encoded_data,
#columns = enc.get_feature_names(['Geography']))

In [10]:
#use feature creation to make a ratio between age and tenure
starter_df['TenureByAge'] = (starter_df['Tenure'] / starter_df['Age']).round(2)
starter_df.drop(columns=['Tenure', 'Age'], inplace=True)

In [11]:
#Remove geography. We are just doing France.
starter_df = starter_df[starter_df.Geography != 'Spain']
starter_df = starter_df[starter_df.Geography != 'Germany']
starter_df.drop(columns=['Geography'], inplace=True)

In [12]:
"""
encoded_df = pd.concat(
    [
        encoded_df_rough,
        starter_df
    ],
    axis=1
)
encoded_df.sample(10)
"""

'\nencoded_df = pd.concat(\n    [\n        encoded_df_rough,\n        starter_df\n    ],\n    axis=1\n)\nencoded_df.sample(10)\n'

In [13]:
X = starter_df.drop(columns= 'Exited')

In [14]:
y = starter_df['Exited']

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [16]:
starter_df['Exited'].value_counts()

0    4204
1     810
Name: Exited, dtype: int64

In [17]:
#Use SMOTE to add synthetic data and balance our target feature value count
oversample = SMOTE()
X_train, y_train = oversample.fit_resample(X_train, y_train)

In [18]:
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [19]:
#Choose a model
#clf = SVC()
#clf = LogisticRegression()
clf = RandomForestClassifier()

In [20]:
#Fit the model on the training data
clf.fit(X_train, y_train)

RandomForestClassifier()

In [21]:
#Make predictions on the test data
test_predictions = clf.predict(X_test)

In [22]:
accuracy = accuracy_score(test_predictions, y_test)
print("Accuracy: {:.2f}%".format(accuracy * 100))

Accuracy: 80.46%


In [23]:
#Create a dataframe showing predictions vs test
comparison = pd.DataFrame({"Testing Data Predictions": test_predictions,
    "Testing Data Actual Targets": y_test})
comparison = comparison.sort_index(ascending=True)
comparison.tail(5)

,Testing Data Predictions,Testing Data Actual Targets
9972,1,0
9977,0,0
9983,1,0
9985,0,0
9996,0,0


In [24]:
#Generate a test matrix
test_matrix = confusion_matrix(y_test, test_predictions)
print(test_matrix)

[[926 120]
 [125  83]]


In [25]:
# Create a testing classifiction report
testing_report = classification_report(y_test, test_predictions)

# Print the testing classification report
print(testing_report)

              precision    recall  f1-score   support

           0       0.88      0.89      0.88      1046
           1       0.41      0.40      0.40       208

    accuracy                           0.80      1254
   macro avg       0.64      0.64      0.64      1254
weighted avg       0.80      0.80      0.80      1254

